In [4]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
import json
import numpy as np
from PIL import Image
import os
import glob
import random
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from sklearn.model_selection import train_test_split

In [6]:
gamma_values = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8]  # Gamma values

def transform(img):
    return img

def gamma_correction(img, gamma):
    return TF.adjust_gamma(img, gamma)

def random_flip(img):
    flip_type = random.choice(["horizontal", "vertical", "both", "none"])

    if flip_type == "horizontal":
        return TF.hflip(img)
    elif flip_type == "vertical":
        return TF.vflip(img)
    elif flip_type == "both":
        return TF.hflip(TF.vflip(img))
    return img  # No flip
    
# 데이터셋 클래스 정의
class BleedingDataset(Dataset):
    def __init__(self, image_files, transform=None, augmentation=False):
        self.image_paths = image_files
        self.json_paths = sorted([f.replace('.jpeg', '.json').replace('.png', '.json') for f in self.image_paths])
        self.transform = transform
        self.augmentation = augmentation
        self.transform_image = transform_image
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 원본 이미지 로드
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        # JSON 파일 로드
        json_path = self.json_paths[idx]
        with open(json_path, 'r') as f:
            data = json.load(f)

        # 빈 마스크 생성
        mask = np.zeros((data["imageHeight"], data["imageWidth"]), dtype=np.uint8)

        # 출혈(BF) 영역 폴리곤 마스크 생성
        for shape in data["shapes"]:
            if shape["label"] == "BF":
                points = np.array(shape["points"], dtype=np.int32)
                cv2.fillPoly(mask, [points], 255)  # 출혈 영역을 255로 설정

        
        # PIL 이미지 변환 후 Tensor 변환
        mask = Image.fromarray(mask)
        
        image = self.transform_image(image)
        mask = self.transform_mask(mask)
            
        if self.augmentation:
            gamma = random.choice(gamma_values)  # 랜덤한 gamma 값 선택
            image = gamma_correction(image, gamma)
            #mask = gamma_correction(mask, gamma)

        return image, mask

# 데이터 변환 정의
transform_image = transforms.Compose([
    transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.BILINEAR),  # 일반 이미지용,
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.NEAREST),  # 일반 이미지용,
    transforms.ToTensor(),
])

# Dice Loss 정의
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    intersection = (pred_flat * target_flat).sum()
    return 1 - (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)

    import torch.nn.functional as F

def focal_loss(pred, target):
    pred = torch.sigmoid(pred)
    bce_loss = F.binary_cross_entropy(pred, target, reduction='none')
    focal_loss = 1 * (1 - torch.exp(-bce_loss)) ** 2 * bce_loss
    return focal_loss.mean()

def iou_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    iou = (intersection + smooth) / (union + smooth)
    return 1 - iou  # IoU가 높을수록 좋기 때문에, Loss는 1에서 뺌

def loss_fn(pred, target):
    return dice_loss(pred, target) + focal_loss(pred, target) + iou_loss(pred, target)


In [7]:
image_dir = "0014_spine_endoscope_data/"
image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpeg', '.png'))])  # 이미지 파일 리스트

# ✅ Train Test Split (85:15 비율)
train_images, test_images = train_test_split(image_files, test_size=0.15, random_state=42)

# train 데이터셋 및 DataLoader 생성
train_dataset = BleedingDataset(train_images, transform=transform, augmentation=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# test 데이터셋 및 DataLoader 생성
test_dataset = BleedingDataset(test_images, transform=transform, augmentation=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

KeyboardInterrupt: 

In [ ]:

# 모델 로드 (ResNet50 기반)
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)

# 출력 채널 변경 (COCO 클래스 → 출혈 탐지 클래스 1개)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

# model.load_state_dict(torch.load("deeplabv3_bleeding.pth"))

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)["out"]  # DeepLabV3의 출력 가져오기
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# 모델 저장
torch.save(model.state_dict(), "deeplabv3_bleeding.pth")
print("모델 저장 완료!")


In [9]:
# 모델 로드 (ResNet50 기반)
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)

# 출력 채널 변경 (COCO 클래스 → 출혈 탐지 클래스 1개)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

model.load_state_dict(torch.load("deeplabv3_bleeding.pth"))

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

output_dir = "test_results/"
os.makedirs(output_dir, exist_ok=True)

total_num = 0
for images, masks in test_dataloader:
    images, masks = images.to(device), masks.to(device)
    optimizer.zero_grad()
    
    with torch.no_grad():
        preds = model(images)["out"]  # DeepLabV3의 출력 가져오기
        
    for image, mask, pred in zip(images, masks, preds):
        total_num += 1
        
        # 📌 후처리 (Sigmoid + Threshold)
        pred_mask = torch.sigmoid(pred).cpu().numpy()  # (Batch, 1, 512, 512)
        pred_mask = (pred_mask > 0.5).astype(np.uint8)  # 이진화
        
        
        
        # 원본 이미지, 마스크 변환
        original_image = image.cpu().numpy().transpose(1,2,0)
        original_image = (original_image * 255).astype(np.uint8)  # 정규화 해제
        original_mask = (mask.squeeze().cpu().numpy() > 0).astype(np.uint8)  # GT 마스크
        pred_mask = pred_mask.squeeze()  # 모델 예측 마스크


        # ✅ 컬러맵 적용 (GT = Green, Pred = Red, Overlap = Yellow)
        overlay = np.array(original_image)
        overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

        green = [0, 255, 0]  # Ground Truth (GT) - Green
        red = [0, 0, 255]  # Prediction - Red
        yellow = [0, 255, 255]  # Overlapping - Yellow

        mask_layer = np.zeros_like(overlay, dtype=np.uint8)
        mask_layer[original_mask == 1] = green  # GT
        mask_layer[pred_mask == 1] = red  # Prediction
        mask_layer[(original_mask == 1) & (pred_mask == 1)] = yellow  # Overlap

        # ✅ 최종 합성
        blended = cv2.addWeighted(overlay, 0.7, mask_layer, 0.5, 0)

        # ✅ 저장
        filename = f"output_{total_num}.png"
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, blended)
        
print("test completed")

TypeError: BleedingDataset.__init__() got an unexpected keyword argument 'augmentation'